# 기사 제목을 가지고 긍정 / 부정 / 중립으로 분류하는 모델 만들어보기

## 1. 학습데이터, 테스트데이터 만들기
먼저 모델을 만들고 나서 학습을 시킬 데이터를 만들기 위해 네이버에서 몇몇의 기업을 선정하고

그 기업에 대한 기사의 제목을 크롤링하였습니다.

 

크롤링 후에 학습시 필요한 긍정, 부정, 중립을 나타내는 label이 있었어야했는데

손으로 일일이 부정, 긍정, 중립 이 세 가지로 label을 붙이려고 하다보니

눈도 아프고 비슷한 내용임에도 불구하고 앞에서는 긍정으로 했다가 뒤에서는 중립으로 표기하는 등의 문제가 있었습니다.

 

이를 컴퓨터가 긍정적인 단어, 부정적인 단어가 포함되어있는지 여부를 확인하여

자동으로 라벨을 붙여주면 편할 것 같아 그렇게 만들어 보았습니다.


[negative_words_self.txt](data/negative_words_self.txt)


[positive_words_self.txt](positive_words_self.txt)



먼저, 긍정적인 단어, 부정적인 단어가 포함된 txt파일을 각각 만들어주었습니다.

뉴스 기사를 보며 만든 긍정적인 단어, 부정적인 단어 모음입니다.

단어는 생각나는대로 계속 추가하고자합니다.

 

코드에서는 이 단어들을 파일에서 positive, negative라는 list로 받아와서

두 개의 list를 합쳐 posneg라는 list를 만들고 크롤링해오는 단어에서 posneg안에 있는 단어가 포함되어있으면 긍정, 부정 라벨을 붙여주고

포함되어있지 않으면 그냥 중립인 0의 상태로 그대로 두도록 만들어 보았습니다.


### 파일에서 단어를 불러와 posneg리스트를 만드는 코드

In [1]:
import codecs 

positive = [] 
negative = [] 
posneg = []

pos = codecs.open("data/positive_words_self.txt", 'rb', encoding='UTF-8')

while True:
    line = pos.readline() 
    line = line.replace('\n', '') 
    positive.append(line) 
    posneg.append(line) 
    
    if not line: 
        break 
        
pos.close()

neg = codecs.open("data/negative_words_self.txt", 'rb', encoding='UTF-8') 

while True: 
    line = neg.readline() 
    line = line.replace('\n', '') 
    negative.append(line) 
    posneg.append(line) 
    
    if not line: break 
        
neg.close()

크롤링한 기사 제목과 기사 제목과 posneg를 활용하여 만든 긍정(1), 부정(-1), 중립(0)라벨 정보를 가지는 dataframe을 만드는 함수

(예시 : 네이버에서 버거킹으로 검색하여 나온 기사 4,000개 제목과 각각 제목의 긍정, 부정, 중립 라벨 생성)

In [8]:
import requests 
from bs4 import BeautifulSoup 
import re 
import pandas as pd 

label = [0] * 4000 
my_title_dic = {"title":[], "label":label} 

j = 0 

for i in range(400): 
    num = i * 10 + 1 
    # bhc # url = "https://search.naver.com/search.naver?&where=news&query=bhc&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=23&start=" + str(num) 
    # 아오리라멘 
    # url2 = "https://search.naver.com/search.naver?&where=news&query=%EC%95%84%EC%98%A4%EB%A6%AC%EB%9D%BC%EB%A9%98&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=34&start=" + str(num) 
    
    # 버거킹 
    url3 = "https://search.naver.com/search.naver?&where=news&query=%EB%B2%84%EA%B1%B0%ED%82%B9&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=23&start=" + str(num) 
    
    req = requests.get(url3) 
    
    soup = BeautifulSoup(req.text, 'lxml') 
    titles = soup.select("a._sp_each_title") 
    
    for title in titles:
        
        title_data = title.text 
        title_data = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…\"\“》]', '', title_data) 
        my_title_dic['title'].append(title_data) 
        
        for i in range(len(posneg)): 
            posflag = False 
            negflag = False 
            
            if i < (len(positive)-1): 
                
                # print(title_data.find(posneg[i])) 
                if title_data.find(posneg[i]) != -1: 
                    posflag = True 
                    print(i, "positive?","테스트 : ",title_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i, title_data) 
                    break 
                    
            if i > (len(positive)-2): 
                
                if title_data.find(posneg[i]) != -1: 
                    negflag = True 
                    print(i, "negative?","테스트 : ",title_data.find(posneg[i]),"비교단어 : ", posneg[i], "인덱스 : ", i, title_data) 
                    break 
                    
        if posflag == True: 
            label[j] = 1 
            # print("positive", j) 
        elif negflag == True: 
            label[j] = -1 
            # print("negative", j) 
        elif negflag == False and posflag == False: 
            label[j] = 0 
            # print("objective", j) 
            
        j = j + 1 
        
    my_title_dic['label'] = label 
    my_title_df = pd.DataFrame(my_title_dic)



46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 햄버거 2개 8000원 행사
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹의 검은빵 버거는 끼리끼리 시너지
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 어떤 버거를 선택해도 2개에 8000원
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹 킹스초이스King’s Choice 프로모션 실시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 침체됐던 햄버거시장 MZ세대 공략으로 회복세
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 위기가 기회코로나 뚫고 사업 확장하는 프랜차이즈
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 코로나19 시대 배달 음식은 일상
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 먹템 흑맥주향 1도 안나네 짠맛 강한 버거킹 기네스와퍼
15 positive? 테스트 :  38 비교단어 :  출시 인덱스 :  15 버거킹 더 맛있게 돌아온 미국 남부의 매콤한 맛 뉴올리언스 치킨버거 출시
46 negative? 테스트 :  0 비교단어 :   인덱스 :  46 버거킹이 헬싱키에서 맥도날드와의 사랑을 선언하다


ValueError: arrays must all be same length

In [6]:
posneg

['MOU',
 '제휴',
 '주목',
 '호응',
 '돌파',
 '이목',
 '수상',
 '입점',
 '인기',
 '열풍',
 '진화',
 '대박',
 '순항',
 '유치',
 '1위',
 '출시',
 '선보여',
 '오픈',
 '팝업',
 '돌풍',
 '팝업스토어',
 '인싸',
 '줄서서',
 '인기',
 '대세',
 '트렌드',
 '불티',
 '진출',
 '부상',
 '체결',
 '증가',
 '봉사',
 '기부',
 '신메뉴',
 '신제품',
 '신상',
 '최고',
 '새로운',
 '편한',
 '미소',
 '맛집',
 '착한가게',
 '캠패인',
 '순항',
 '착한',
 '제작지원',
 '']